In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import glob
import shutil
import random
import psutil
import random

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-05-05 18:17:41.499315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-05 18:17:42.423584: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
from dataloaders.OptimizedDataGenerator import OptimizedDataGenerator
from loss import *
from models.models import *

2025-05-12 13:26:45.745560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747074405.765701 4004372 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747074405.772097 4004372 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747074405.787944 4004372 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747074405.787969 4004372 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747074405.787972 4004372 computation_placer.cc:177] computation placer alr

ModuleNotFoundError: No module named 'tensorflow_probability'

#### Set random seed for reproducibility

In [3]:
seed = 10
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

#### Scaling Lists for Different Pixel Pitches (dataset_2s):
* 100x25x100 um:  [150.0, 37.5, 10.0, 1.22]
* 50x25x100 um:   [75.0, 37.5, 10.0, 1.22]
* 50x20x100 um:   [75.0, 30.0, 10.0, 1.22]
* 50x15x100 um:   [75.0, 22.5, 10.0, 1.22]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 1.22]
* 50x10x100 um:   [75.0, 15.0, 10.0, 1.22]

#### Scaling Lists for Different Pixel Pitches (dataset_3sr):
* 100x25x100 um:  [150.0, 37.5, 10.0, 10.0]
* 50x25x100 um:   [75.0, 37.5, 10.0, 10.0]
* 50x20x100 um:   [75.0, 30.0, 10.0, 10.0]
* 50x15x100 um:   [75.0, 22.5, 10.0, 10.0]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 10.0]
* 50x10x100 um:   [75.0, 15.0, 10.0, 10.0]

#### Scaling Lists for Different Pixel Pitches (dataset_3sr, |cot$\beta$| $\leq$ 1.5 preselection):
* 100x25x100 um:  [150.0, 37.5, 10.0, 1.5], 
* 50x25x100 um:   [75.0, 37.5, 10.0, 1.5],
* 50x20x100 um:   [75.0, 30.0, 10.0, 1.5]
* 50x15x100 um:   [75.0, 22.5, 10.0, 1.5]
* 50x12.5x100 um: [75.0, 18.75, 10.0, 1.5]
* 50x10x100 um:   [75.0, 15.0, 10.0, 1.5]

In [4]:
batch_size = 5000
val_batch_size = 5000
train_file_size = 20
val_file_size = 20

# Specify what directory to save the tfrecords
tfrecords_dir_train = '/data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_2s_50x12P5_2t_train/'
tfrecords_dir_val = '/data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_2s_50x12P5_2t_val/'

training_generator = OptimizedDataGenerator(
    data_directory_path = '/data/dajiang/smart-pixels/dataset_2s/dataset_2s_50x12P5_parquets/unflipped/',
    labels_directory_path = '/data/dajiang/smart-pixels/dataset_2s/dataset_2s_50x12P5_parquets/unflipped/',
    is_directory_recursive = False,
    file_type = 'parquet',
    data_format = '3D',
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= False,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.22],
    input_shape = (2,13,21),
    transpose = (0,2,3,1),
    files_from_end = True,
    shuffle = True,

    #load_from_tfrecords_dir = '/data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_2s_50x12P5_2t_train',
    tfrecords_dir = tfrecords_dir_train,
    use_time_stamps = [0,19], #-1
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = seed,
    quantize = True # Quantization ON
)

validation_generator = OptimizedDataGenerator(
    data_directory_path = '/data/dajiang/smart-pixels/dataset_3sr/dataset_3sr_50x12P5_cotBeta1P5_quantize_manual_400_1515_2971_parquets/unflipped/',
    labels_directory_path = '/data/dajiang/smart-pixels/dataset_3sr/dataset_3sr_50x12P5_cotBeta1P5_quantize_manual_400_1515_2971_parquets/unflipped/',
    is_directory_recursive = False,
    file_type = 'parquet',
    data_format = '3D',
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= False,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    scaling_list = [75.0, 18.75, 10.0, 1.5],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end = False,
    shuffle = True,

    #load_from_tfrecords_dir = '/data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_2s_50x12P5_2t_val',
    tfrecords_dir = tfrecords_dir_val,
    use_time_stamps = -1, #-1
    max_workers = 1, # Don't make this too large (will use up all RAM)
    seed = seed,
    quantize = True # Quantization ON
)

Directory /data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_3sr_50x12P5_20t_bs5000_cotBeta1P5_quantize_manual_400_1515_2971_train does not exist and cannot be removed.


Saving batches as TFRecords: 100%|██████████| 137/137 [02:18<00:00,  1.01s/it]


Directory /data/dajiang/smart-pixels/tfrecords/tfrecords_dataset_3sr_50x12P5_20t_bs5000_cotBeta1P5_quantize_manual_400_1515_2971_val does not exist and cannot be removed.


Saving batches as TFRecords: 100%|██████████| 45/45 [00:52<00:00,  1.16s/it]


In [5]:
# compiles model
model=CreateModel((13,21,20),n_filters=5,pool_size=3)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 20)]      0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         285       
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 3, 6, 5)           0     

In [6]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)

In [ ]:
# training
pitch = '50x12P5'
fingerprint = '%08x' % random.randrange(16**8)
base_dir = '/home/dajiang/smart-pixels-ml/weights/weights_7pitches/dataset_3sr_cotBeta1P5_quantize_manual_400_1515_2971_weights/'
weights_dir = base_dir + 'weights-{}-bs{}-{}-checkpoints'.format(pitch, batch_size, fingerprint)

# create output directories
if os.path.isdir(base_dir):
    os.mkdir(weights_dir)
else:
    os.mkdir(base_dir)
    os.mkdir(weights_dir)
    
checkpoint_filepath = weights_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

print('Model fingerprint: {}'.format(fingerprint))

history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[mcp],
                    epochs=500,
                    shuffle=False, # shuffling now occurs within the data-loader
                    verbose=1)

Model fingerprint: 34c2da80
Epoch 1/500


2025-04-28 17:07:25.109394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2025-04-28 17:07:25.185682: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-04-28 17:07:25.473114: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-04-28 17:07:25.482070: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x7fea4d7a7590
2025-04-28 17:07:25.523114: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd6fd4cb2e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-28 17:07:25.523152: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 1g.5gb, Compute Capability 8.0
2025-04-28 17:07:25.528678: I tensorflow/compiler/mlir/tensorflow/utils/dump

137/137 [==============================] - 51s 346ms/step - loss: 30816.7715 - val_loss: 11970.6221
Epoch 2/500
137/137 [==============================] - 30s 217ms/step - loss: 8432.2686 - val_loss: 5177.7031
Epoch 3/500
137/137 [==============================] - 29s 213ms/step - loss: 4932.2588 - val_loss: 2994.5945
Epoch 4/500
137/137 [==============================] - 28s 207ms/step - loss: 2049.4468 - val_loss: 859.0220
Epoch 5/500
137/137 [==============================] - 29s 211ms/step - loss: 643.0757 - val_loss: 590.7726
Epoch 6/500
137/137 [==============================] - 30s 217ms/step - loss: 259.2581 - val_loss: -274.7324
Epoch 7/500
137/137 [==============================] - 28s 206ms/step - loss: -130.7471 - val_loss: -732.5890
Epoch 8/500
137/137 [==============================] - 29s 212ms/step - loss: -736.2985 - val_loss: -888.9834
Epoch 9/500
137/137 [==============================] - 28s 207ms/step - loss: -1480.1647 - val_loss: -1999.3049
Epoch 10/500
137/137 [